In [ ]:
from src.agents import Agent
from src.utils import normalize, extract_walls_from_geometry
from shapely.geometry import Polygon, Point
import numpy as np
import pandas as pd
import pedpy
import jupedsim as jps
from jupedsim.internal.notebook_utils import animate
from src.parameters import ForceParameters

In [ ]:
from shapely import wkt
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import matplotlib.patches as mpatches
geometry_file = "data/geometry.wkt"
with open(geometry_file, "r") as myfile:
    data = myfile.read().strip()

geometry_collection = wkt.loads(data)
polygons = list(geometry_collection.geoms)  # Unpack main-level geometries

# Extract walkable area
area = polygons[0].geoms[0]
area_no_holes = Polygon(area.exterior.coords)
signs = polygons[0].geoms[0].interiors
# Extract exits (second collection)
exit_polygons = []
if len(polygons) > 1:
    exits = polygons[1]
    exit_polygons = list(exits.geoms)

# Extract distributions (third collection)
distribution_polygons = []
if len(polygons) > 2:
    distributions = polygons[2]
    distribution_polygons = list(distributions.geoms)

print(f"Exits: {len(exit_polygons)} polygons")
print(f"Distributions: {len(distribution_polygons)} polygons")

In [ ]:
def plot_geometry(geometry_collection, positions=None):
    """Plot the geometry collection with different colors for each geometry type."""
    fig, ax = plt.subplots(figsize=(8, 8))
    if geometry_collection.is_empty:
        print("Skipping empty geometry.")
        return
    polygons = list(geometry_collection.geoms)  # Unpack main-level geometries
    area = polygons[0] if len(polygons) > 0 else None
    # Extract exits (second collection)
    if len(polygons) > 1:
        exits = polygons[1]

    # Extract distributions (third collection)
    if len(polygons) > 2:
        distributions = polygons[2]

    for geom in area.geoms:
        x, y = geom.exterior.xy
        plt.fill(x, y, alpha=0.2,  edgecolor="black", facecolor="gray")
    

    for hole in geom.interiors:
        hole_polygon = Polygon(hole)
        centroid = hole_polygon.centroid
        radius = hole_polygon.minimum_rotated_rectangle.length / (2 * np.pi)  # rough approx
        circle = Circle(
            (centroid.x, centroid.y),
            radius=radius,
            edgecolor="blue",
            facecolor="blue",
            linewidth=1.0,
            alpha=0.7
        )
        ax.add_patch(circle)



    for e in exits.geoms:
        x, y = e.exterior.xy
        plt.fill(x, y, alpha=0.3, color="red")

    for d in distributions.geoms:
        x, y = d.exterior.xy
        plt.fill(x, y, alpha=0.3, color="green")

    if positions is not None:
        for position in positions:
            x, y = position[0], position[1]
            plt.plot(x, y, "o", color="blue")

    legend_elements = {
        "area": mpatches.Patch(color="gray", label="Walkable Area"),
        "signs": mpatches.Patch(color="blue", label="Signs"),
        "exits": mpatches.Patch(color="red", label="Exits"),
        "distribution": mpatches.Patch(color="green", label="Distribution Zones"),
    }
    handles = [
        legend_elements[key]
        for key in ["area", "signs", "exits", "distribution"]
        if key in legend_elements
    ]
    ax.legend(
        handles=handles,
        loc="upper center",
        bbox_to_anchor=(0.5, 1.05),
        ncol=len(handles),
        frameon=False,
    )
    ax.set_xlabel("X [m]")
    ax.set_ylabel("Y [m]")
    plt.show()
    
plot_geometry(geometry_collection)    

In [ ]:
params = ForceParameters(
    a=0.2,
    wall_distance=1,
    wall_strength_always=2.,
    wall_strength_into=3.,
    exit_strength=1,
    q1=0.5,
    q2=0.2, # q2< q1
    eta_mem=1,
    eta_sign=2,
    hi=0.0,
    sign_vision_radius= 20.5,
    fov_angle= np.pi * 2 / 3,  # 120 degrees
    exit_domain_radius = 14.0
)

In [ ]:
# Run simulation
dt, steps = 0.01, 1000
num_agents = 20
distribution = distribution_polygons[0]
walls = extract_walls_from_geometry(area_no_holes)
exits = exit_polygons
positions = jps.distribute_by_number(
    polygon=Polygon(distribution),
    number_of_agents=num_agents,
    distance_to_agents=0.4,
    distance_to_polygon=0.2,
)

agents = []
for position in positions:
    agent = Agent(position=position, velocity=[-1.0, 0.0], params=params)
    agents.append(agent)

h_i = np.array([params.hi]*2)


active_agents = list(range(len(agents)))  # keep track of active agent indices

# Simulation
trajectories = []
for frame in range(steps):
    # Store next state separately to avoid updates affecting other agents' inputs
    next_positions = []
    next_velocities = []
    # for agent1 in agents: calculate the influnce of all other agents
    print(f"{frame = }", end="\r")
    for i in active_agents:
        agent = agents[i]
        # Gather others (positions and velocities)
        others = [(agents[j].x, agents[j].v) for j in active_agents if j != i]

        # Compute forces and update
        agent.compute_forces(others, walls, signs=signs, exits=exits, h_i=h_i)
        agent.update(dt)

        o = normalize(agent.v)
        trajectories.append(
            {
                "frame": frame,
                "id": i,
                "x": agent.x[0],
                "y": agent.x[1],
                "ox": o[0],
                "oy": o[1],
            }
        )

    # Remove agents who have reached the exit
    still_active = []
    for i in active_agents:
        if not Point(agents[i].x).within(exits[0]):
            still_active.append(i)
        
    active_agents = still_active

    # Stop early if all agents are done
    if not active_agents:
        break
# Convert to DataFrame
df = pd.DataFrame(trajectories)

In [ ]:
traj = pedpy.TrajectoryData(data=df, frame_rate=1/dt)
walkable_area = pedpy.WalkableArea(polygon=area_no_holes)
pedpy.plot_trajectories(walkable_area=walkable_area, traj=traj)

In [ ]:
animate(data=traj, area=walkable_area, every_nth_frame=10, title_note="Hirai & Tarui, 1975")

## Debugging

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
w0 = 6.0
w1 = 2.0
d_cutoff = 1.0  # d in the equation

# Distances to wall
d_i = np.linspace(0, 2, 500)  # covers both d_i < d and d_i > d

# Velocity component into the wall
v_wi_positive = 1.0  # into the wall (v_wi > 0)
v_wi_negative = -1.0  # away from the wall (v_wi <= 0)

# Force magnitudes
F_positive = np.where(d_i < d_cutoff,
                      (w0 * v_wi_positive * (d_cutoff - d_i) / d_cutoff + w1),
                      0)

F_negative = np.where(d_i < d_cutoff,
                      w1,
                      0)

# Plotting
plt.figure(figsize=(8, 5))
plt.plot(d_i, F_positive, label=r"$v_{wi} > 0$ (into wall)", color="red")
plt.plot(d_i, F_negative, label=r"$v_{wi} \leq 0$ (away from wall)", color="blue")
plt.axvline(x=d_cutoff, color="gray", linestyle="--", label="cutoff distance $d$")
plt.xlabel(r"Distance to wall $d_i$")
plt.ylabel(r"Force magnitude $F_{wi}$")
plt.title("Wall Force $F_{wi}$ vs. Distance $d_i$")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
